In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

 
global LOC, data_path
LOC = '/home/bkelley/capstone/data_collection/weather'
data_path = '/home/bkelley/capstone/data_collection/weather/data/hourly_weather_with_temp_avg.csv'



if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    # diamonds = sns.load_dataset("diamonds"), for practice see https://www.datacamp.com/tutorial/xgboost-in-python
    data = pd.read_csv(data_path)
    # print(data.dtypes
    # Extract feature and target arrays
    # isolating the features into X and the target into y:
    headers = data.columns
    # print(headers)
    x, y = data.drop('temperature_2m', axis=1), data[['surface_pressure_Pa']]
    cats = x.select_dtypes(exclude=np.number).columns.tolist()
    for col in cats:
        x[col] = x[col].astype('category')
    # print(x.dtypes)
    # Split the data (size=0.25)
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)
    # Create regression matrices
    dtrain_reg = xgb.DMatrix(x_train, y_train, enable_categorical=True)
    dtest_reg = xgb.DMatrix(x_test, y_test, enable_categorical=True)
    # Define hyperparameters
    params = {"objective": "reg:squarederror", "tree_method": "hist"}  # NEED GPU HIST to run off of GPU
    n = 100
    model = xgb.train(
       params=params,
       dtrain=dtrain_reg,
       num_boost_round=n,
    )
    print('done')


NameError: name 'done' is not defined

In [2]:
preds = model.predict(dtest_reg)
# print(len(preds))
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 200
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=5, # print every 5
   # Activate early stopping
   early_stopping_rounds=50
)


print('done')


RMSE of the base model: 27.297
[0]	train-rmse:494.89840	validation-rmse:492.68427
[5]	train-rmse:85.22572	validation-rmse:87.90217
[10]	train-rmse:20.22505	validation-rmse:31.07321
[15]	train-rmse:12.30869	validation-rmse:27.37471
[20]	train-rmse:10.72529	validation-rmse:27.24115
[25]	train-rmse:9.98299	validation-rmse:27.22857
[30]	train-rmse:9.33126	validation-rmse:27.18570
[35]	train-rmse:8.87660	validation-rmse:27.19500
[40]	train-rmse:8.43436	validation-rmse:27.19528
[45]	train-rmse:8.11295	validation-rmse:27.20592
[50]	train-rmse:7.83242	validation-rmse:27.21670
[55]	train-rmse:7.54595	validation-rmse:27.21600
[60]	train-rmse:7.32350	validation-rmse:27.22715
[65]	train-rmse:7.12548	validation-rmse:27.23827
[70]	train-rmse:6.92993	validation-rmse:27.24295
[75]	train-rmse:6.76789	validation-rmse:27.25400
[78]	train-rmse:6.67666	validation-rmse:27.26058
done


In [3]:
# 5 fold classification
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=35
)
results.head()
best_rmse = results['test-rmse-mean'].min()
best_rmse
# np.float64(29.907072535283824)
# re-train with all data (like above) using  xgb.train


np.float64(29.907072535283824)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np
import xgboost as xgb

# Prepare the data
x, y = data.drop('temperature_2m', axis=1), data[['surface_pressure_Pa']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Count occurrences of each class in y_encoded
y_encoded_df = pd.DataFrame(y_encoded, columns=['y_encoded'])
class_counts = y_encoded_df['y_encoded'].value_counts()

# Remove rows where the class count is less than 2
valid_classes = class_counts[class_counts >= 2].index
data_filtered = data[y_encoded_df['y_encoded'].isin(valid_classes)]

# Split the data again
x_filtered, y_filtered = data_filtered.drop('temperature_2m', axis=1), data_filtered[['surface_pressure_Pa']]

# Re-encode y_filtered
y_filtered_encoded = OrdinalEncoder().fit_transform(y_filtered)

# Adjust labels to be in the range [0, num_class)
y_filtered_encoded = y_filtered_encoded - y_filtered_encoded.min()

# Ensure num_class is set correctly based on unique classes
num_class = len(np.unique(y_filtered_encoded))

# Extract text features and convert to categorical
cats = x_filtered.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
    x_filtered[col] = x_filtered[col].astype('category')

# Split the filtered data
x_train, x_test, y_train, y_test = train_test_split(x_filtered, y_filtered_encoded, random_state=1, stratify=y_filtered_encoded)

# Create classification matrices
dtrain_clf = xgb.DMatrix(x_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(x_test, y_test, enable_categorical=True)

# XGBoost parameters
params = {
    "objective": "multi:softprob",  # Softmax for multi-class classification
    "tree_method": "hist",          # Efficient histogram-based tree method
    "num_class": num_class,         # Set to the actual number of classes
}

n = 100  # Number of boosting rounds

# Perform cross-validation
results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

print(results)


In [3]:
# say x, y = data.drop('temperature_2m', axis=1), data[['surface_pressure_Pa']]
# RMSE validatio of ~ 27
print(data['surface_pressure_Pa'].max() - data['surface_pressure_Pa'].min())

8482.426000000007


In [4]:
'''
print(data['surface_pressure_Pa'].max())
print(data['surface_pressure_Pa'].min())
relative rmse = 
'''




104710.24
96227.814
